## cov2m: cov2 + mega-virus genome
```
Lead     : ababaian
Issue    : #41
start    : 2020 41 08
complete : 2020 04 08
files    : s3://serratus-public/seq/cov2m/
```

## Introduction

Minor refinements on cov2r genome:
- Remove `REVERSE` control sequences
- 

Addition of mega0, mega-viral genomes:
- 



## Materials and Methods


In [ ]:
# EC2 Instance Commands:
# Build/Run `serratus-align`container for indexing
sudo yum install -y docker
sudo yum install -y git
sudo service docker start

git clone https://github.com/ababaian/serratus.git; cd serratus/containers
./build_containers

sudo docker run --rm --entrypoint /bin/bash -it serratus-align:latest

In [ ]:
# local bedtools install
wget https://github.com/arq5x/bedtools2/releases/download/v2.29.2/bedtools.static.binary
mv bedtools.static.binary bedtools
chmod 755 bedtools; mv bedtools /usr/bin/

In [ ]:
# Local usearch install
#The clustered database was made with usearch:
wget https://drive5.com/downloads/usearch11.0.667_i86linux32.gz
gzip -dc usearch11.0.667_i86linux32.gz > usearch
chmod 755 usearch; mv usearch /usr/bin/usearch


In [1]:
# Local Dustmasker install
cd /home/serratus/
wget ftp://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ncbi-blast-2.10.0+-x64-linux.tar.gz
tar -xvf ncbi-blast-2.10.0+-x64-linux.tar.gz
cp ncbi-blast-2.10.0+/bin/* /usr/bin/

# Building from source (do not do)
#wget ftp://ftp.ncbi.nih.gov/toolbox/ncbi_tools++/CURRENT/ncbi_cxx--22_0_0.tar.gz
#tar -xvf ncbi_cxx--22_0_0.tar.gz
#
## g++ compiler
#sudo yum groupinstall "Development Tools"
#
#
## NCBI toolkit dependencies
## sudo yum install -y sqlite3
#sudo yum install -y boost lzo libxml2 pcre libxslt \
#  libungif libtiff libpng bzip2
## berkleyDB
#wget https://download.oracle.com/berkeley-db/db-4.7.25.NC.zip
#unzip db-4.7.25.NC.zip; cd db-4.7.25.NC/build_unix/
#../dist/configure
#make; make install
#
## fastCGI
##wget http://www.fastcgi.com/dist/mod_fastcgi-current.tar.gz
##tar -xzvf mod_fastcgi-2.4.6.tar.gz
##cd mod_fastcgi-2.4.6
##cp Makefile.AP2 Makefile
#
#cd /home/serratus/ncbi_cxx--22_0_0
#./configure
#make; make install


[sudo] password for artem: 


This is a 'patch' to `cov2` so those pruned sequences
are used as basis.

See: [cov2 pangenome notebook](./200420_cov2_pangenome.ipynb)


In [ ]:
# bash inside `serratus-align`
mkdir cov2; cd cov2

# Start from cov2 - post 99% identity filtering
aws s3 cp s3://serratus-public/seq/cov2r/cov.full.headers.gz ./
aws s3 cp s3://serratus-public/seq/cov2r/cov0.id99.fa ./


In [ ]:
# Remove Accessions on blacklist
# Strips headers to accessions only
# wc -l cov0.id99.fa
# 514753 cov0.id99.fa

seqkit grep cov0.id99.fa -i -r -v \
  -p KC786228 -p AX191447 -p AX191449 \
  -p FB764528 -p HV449436 -p CS382036 \
  > cov0.id99.bl.fa

# wc -l cov0.id99.bl.fa

seqkit grep cov0.id99.bl.fa -i -r -v \
  -p FV537213 -p AY204705 -p AF209745 \
  -p AY204704 \
  > cov0.id99.bl.2.fa



In [ ]:
# mega0 viral mega-genome
# remove descriptors; keep accessions
aws s3 cp s3://serratus-public/seq/mega0/mega0.fa ./
grep "^>" mega0.fa > mega0.full.headers
gzip mega0.full.headers

# remove duplicates, strip headers to accessions
# 0 removed
seqkit rmdup -s -i -D mega0.duplicates mega0.fa |
seqkit seq -i -m 10 - > mega0.rmdup.fa


In [ ]:
cat cov0.id99.bl.2.fa mega0.rmdup.fa > cov2m.unmasked.fa

In [ ]:
INPUTFA='cov2m.unmasked.fa'

# Create polyNT masks (10-X seed)
seqkit locate --bed -i -m 0 -p 'AAAAAAAAAA' $INPUTFA > poly10.bed
  bedtools sort -chrThenSizeA -i poly10.bed > poly10.sort.bed
  bedtools merge -s -i poly10.sort.bed > polyAT.mask.bed

seqkit locate --bed -i -m 0 -p 'GGGGGGGGGG' $INPUTFA > poly10.bed
  bedtools sort -chrThenSizeA -i poly10.bed > poly10.sort.bed
  bedtools merge -s -i poly10.sort.bed > polyGC.mask.bed

cat polyAT.mask.bed polyGC.mask.bed > \
  polyNT.bed

rm polyAT.mask.bed polyGC.mask.bed poly10.bed poly10.sort.bed

In [ ]:
# Manually set blacklisted regions
echo -e "JB181528.1\t3111\t3307" >> blacklist.bed
echo -e "CS460762.1\t37177\t37211" >> blacklist.bed
echo -e "CS460762.1\t30166\t30243" >> blacklist.bed
echo -e "CS480537.1\t37170\t37220" >> blacklist.bed
echo -e "CS480537.1\t30166\t30241" >> blacklist.bed
echo -e "MK562374.1\t474\t542" >> blacklist.bed
echo -e "DL231478.1\t43\t2296" >> blacklist.bed

# new for cov2m
echo -e "KC786228.1\t49\t191 " >> blacklist.bed

In [ ]:
INPUTFA='cov2m.unmasked.fa'

# Soft mask low complexity regions via dustmasker
dustmasker -in $INPUTFA \
  -window 30 -outfmt interval \
  -out cov2m.dust
# output ... 
# >KF600647.1
# 2840 - 2846
# 30101 - 30114
# >MK129253.1
# 2845 - 2851
# 30106 - 30149
# >CS480537.1
# 22517 - 22524
# 22569 - 22575
# 29227 - 29246
# ...

# Convert interval dust file to bed file
while read -r line; do
  if [ $(echo $line | head -c 1) = ">" ]; then
    header=$( echo $line | sed 's/>//g' - )
  else
    start=$(echo $line | cut -f1 -d' ' -)
    end=$(echo $line | cut -f3 -d' ' - )
    echo -e "$header\t$start\t$end" >> dust.bed
  fi
done < cov2m.dust

cat dust.bed >> 

In [ ]:
# Blacklist, polyNT and dust mask

cat polyNT.bed blacklist.bed dust.bed > mask.regions.bed

rm polyNT.bed blacklist.bed dust.bed

In [ ]:
INPUTFA='cov2m.unmasked.fa'

# cov2 pan-genome
# Soft-masked pan-genome
bedtools maskfasta -fi $INPUTFA \
  -bed mask.regions.bed -fo cov2m.softmasked.fa -soft
 
# Hard-masked pan-genome
bedtools maskfasta -fi $INPUTFA \
  -bed mask.regions.bed -fo cov2m.fa -mc N

In [ ]:
# Count each fasta file
wc -l *.fa 

```
   514256 cov0.id99.bl.2.fa
   514753 cov0.id99.bl.fa
   514753 cov0.id99.fa
   593294 cov2m.fa
   593294 cov2m.softmasked.fa
   593294 cov2m.unmasked.fa
    59407 mega0.fa
    79038 mega0.rmdup.fa
  3462089 total
```

In [ ]:
# Remove intermediates
rm cov0* mega0.fa mega0.rmdup.fa

# Compress stuff we don't need immediatly
gzip cov2m.dust cov2m.softmasked.fa cov2m.unmasked.fa

In [ ]:
# Build bowtie2 + faidx index for cov1r.fa
bowtie2-build --threads 4 --seed 666 cov2m.fa cov2m
samtools faidx cov2m.fa

```
drwxr-xr-x 2 root     root     4.0K May  9 21:55 .
drwx------ 1 serratus serratus  110 May  9 20:26 ..
-rw-r--r-- 1 root     root     322K Apr 21 21:44 cov.full.headers.gz
-rw-r--r-- 1 root     root      16M May  9 21:52 cov2m.1.bt2
-rw-r--r-- 1 root     root     8.3M May  9 21:52 cov2m.2.bt2
-rw-r--r-- 1 root     root     138K May  9 21:52 cov2m.3.bt2
-rw-r--r-- 1 root     root     8.3M May  9 21:52 cov2m.4.bt2
-rw-r--r-- 1 root     root      63K May  9 21:36 cov2m.dust.gz
-rw-r--r-- 1 root     root      34M May  9 21:47 cov2m.fa
-rw-r--r-- 1 root     root     244K May  9 21:55 cov2m.fa.fai
-rw-r--r-- 1 root     root      16M May  9 21:53 cov2m.rev.1.bt2
-rw-r--r-- 1 root     root     8.3M May  9 21:53 cov2m.rev.2.bt2
-rw-r--r-- 1 root     root     9.5M May  9 21:47 cov2m.softmasked.fa.gz
-rw-r--r-- 1 root     root     9.5M May  9 21:34 cov2m.unmasked.fa.gz
-rw-r--r-- 1 root     root     129K May  9 21:45 mask.regions.bed
-rw-r--r-- 1 root     root     5.1K May  9 21:30 mega0.full.headers.gz
```

In [ ]:
# Upload to s3 public access area
aws s3 sync ./ s3://serratus-public/seq/cov2m/

## Results & Discussion

The `cov2m` pan-coronavirus + viral mega-genome and it's respective `bowtie2` index is prepared.

Note, seqkit destroys the original headers so now each 'chromosome' or input sequence is referred to by it's accession ID only. As a remedy the `cov.full.headers.gz` text file is available.

#### Downloading cov2 sequences

`aws s3 cp --recursive s3://serratus-public/seq/cov2r/ ./`

